In [ ]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline 
import matplotlib
matplotlib.rcParams["figure.figsize"]=(20,10)

In [ ]:
#df1=pd.read_csv("../input/house-rent-prices-of-metropolitan-cities-in-india/Ahmedabad_rent.csv")
#df2=pd.read_csv("../input/house-rent-prices-of-metropolitan-cities-in-india/Bangalore_rent.csv")
#df3=pd.read_csv("../input/house-rent-prices-of-metropolitan-cities-in-india/Chennai_rent.csv")
#df4=pd.read_csv("../input/house-rent-prices-of-metropolitan-cities-in-india/Delhi_rent.csv")
#df5=pd.read_csv("../input/house-rent-prices-of-metropolitan-cities-in-india/Hyderabad_rent.csv")
#df6=pd.read_csv("../input/house-rent-prices-of-metropolitan-cities-in-india/Kolkata_rent.csv")
#df7=pd.read_csv("../input/house-rent-prices-of-metropolitan-cities-in-india/Mumbai_rent.csv")
df8=pd.read_csv("../input/house-rent-prices-of-metropolitan-cities-in-india/Pune_rent.csv")

In [ ]:
df8.head()

In [ ]:
df8["city"]="Pune"

In [ ]:
#data = pd.concat([df1, df2, df3, df4, df5, df6, df7, df8], axis=0, join='inner')
#data.head()

In [ ]:
df8.shape

In [ ]:
df8.groupby('property_type')['property_type'].agg('count')

In [ ]:
df8.isnull().sum()

In [ ]:
df8=df8.dropna()
df8.isnull().sum()

In [ ]:
df8[df8.bedroom>10]

In [ ]:
df8.area.unique

In [ ]:
df8.price.dtype

In [ ]:
df8.price.unique()

In [ ]:
df8['price']=df8.price.apply(lambda x: str(x).replace(',', ''))
#data['Company_txt'] = data.Company_txt.apply(lambda x: str(x).replace('\n', ''))
df8.price

In [ ]:
#df8['price']=df8.price.apply(lambda x:int(x))
df8['price'] = df8['price'].astype(float)
#df8['price']*100000

In [ ]:
df8['price_per_sqft']=df8['price']*100000/df8['area']
df8['price_per_sqft']

In [ ]:
len(df8.locality.unique())

In [ ]:
df8['locality']=df8['locality'].apply(lambda x: x.strip())
location_stats=df8.groupby('locality')['locality'].agg('count').sort_values(ascending=False)
location_stats

In [ ]:
loc_less_than_10=location_stats[location_stats<=10]
len(loc_less_than_10)

In [ ]:
len(df8.locality.unique())

In [ ]:
df8['locality']=df8['locality'].apply(lambda x: 'other' if x in loc_less_than_10 else x)
len(df8.locality.unique())

In [ ]:
df8.head()

In [ ]:
#Assuming each bedroom is atleast 300 sqft
df8.area_per_bedroom=df8.area/df8.bedroom
df8.area_per_bedroom

In [ ]:
df8.head()

In [ ]:
df8.outliers=df8[df8.area_per_bedroom<300]
df8.outliers

In [ ]:
#df6=df8[~df8.outliers]
#df8.drop([df8.outliers], inplace = True )
df8=df8[(df8.area/df8.bedroom)>=300]
#df8.shape
df8.head()

In [ ]:
df8.price_per_sqft.describe()

In [ ]:
def remove_pps_outliers(df):
    df_out=pd.DataFrame()
    for key,subdf in df.groupby('locality'):
        m=np.mean(subdf.price_per_sqft)
        st=np.std(subdf.price_per_sqft)
        reduced_df=subdf[(subdf.price_per_sqft>(m-st))&(subdf.price_per_sqft<=(m+st))]
        df_out=pd.concat([df_out,reduced_df],ignore_index=True)
    return df_out

df8=remove_pps_outliers(df8)
df8.shape

In [ ]:
def plot_scatter_chart(df,location):
    bhk2=df[(df.locality==location) & (df.bedroom==2)]
    bhk3=df[(df.locality==location) & (df.bedroom==3)]
    matplotlib.rcParams['figure.figsize']=(15,10)
    plt.scatter(bhk2.area,bhk2.price,color='blue',label='2 BHK', s=50)
    plt.scatter(bhk3.area,bhk3.price,color='green',label='3 BHK', s=50)
    plt.xlabel("Total Square Feet Area")
    plt.ylabel("Price Per Square Feet")
    plt.title(location)
    plt.legend()
    
plot_scatter_chart(df8,'Hadapsar')

    

In [ ]:
import matplotlib
matplotlib.rcParams["figure.figsize"]=(20,10)
plt.hist(df8.price_per_sqft,rwidth=0.8)
plt.xlabel("Price Per Square Feet")
plt.ylabel("Count")

In [ ]:
df8.bathroom.unique()

In [ ]:
#df8.outlier_bathrooms=df8[(df8.bathroom=='East facing') | (df8.bathroom=='West facing') | (df8.bathroom=='Grfloor') | (df8.bathroom=='11 of 11floor') | (df8.bathroom=='1 of 2floor') | (df8.bathroom=='NorthEast facing') | (df8.bathroom=='North facing')]
df8.drop(df8[(df8.bathroom=='East facing') | (df8.bathroom=='West facing') | (df8.bathroom=='Grfloor') | (df8.bathroom=='11 of 11floor') | (df8.bathroom=='1 of 2floor') | (df8.bathroom=='NorthEast facing') | (df8.bathroom=='North facing') ].index,inplace=True)
#df8.bath.unique()
#data=df8[df8.]
#df8.
#df8.outlier_bathrooms

In [ ]:
df8.bathroom=df8['bathroom'].apply(lambda x:x.split(' ')[0])
df8.bathroom.unique()
#df8=pd.concat([df8,df8.bath],axis='columns')
#.drop('other', axis='columns')


In [ ]:
df8.columns

In [ ]:
df8.bathroom

In [ ]:
df8

In [ ]:
#df8[int(df8.bath)>df8.bedroom+2]
#df8['bath'] = df8['bath'].astype(int)
pd.to_numeric(df8.bathroom, errors='coerce')

In [ ]:
dummy=pd.get_dummies(df8.locality)
dummy

In [ ]:
df8=pd.concat([df8,dummy.drop('other', axis='columns')],axis='columns')
df8.head()

In [ ]:
df8=df8.drop(['seller_type','price_per_sqft','layout_type','locality','furnish_type','city','property_type','bathroom'],axis='columns')

In [ ]:
df8.shape

In [ ]:
X=df8.drop('price',axis='columns')
X

In [ ]:
y=df8.price
y

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train, y_test=train_test_split(X,y,test_size=0.2,random_state=10)

from sklearn.linear_model import LinearRegression
lr_clf = LinearRegression()
lr_clf.fit(X_train,y_train)
lr_clf.score(X_test,y_test)

In [ ]:
from sklearn.model_selection import ShuffleSplit
from sklearn.model_selection import cross_val_score
cv=ShuffleSplit(n_splits=5,test_size=0.2,random_state=0)
cross_val_score(LinearRegression(),X,y,cv=cv)

In [ ]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import Lasso
from sklearn.tree import DecisionTreeRegressor

def find_best_model_using_gridsearchcv(X,y):
    algos={
        'linear_regression': {
            'model': LinearRegression(),
            'params':{
                'normalize':[True, False]
            }
        },
        'lasso': {
            'model': Lasso(),
            'params':{
                'alpha':[1, 2],
                'selection':['random', 'cyclic']
            }
        },
        'decision_tree': {
            'model': DecisionTreeRegressor(),
            'params':{
                'criterion':['mse', 'friedman_mse'],
                'splitter':['best', 'random']
            }
        }
    }
    
    scores=[]
    cv=ShuffleSplit(n_splits=5,test_size=0.2,random_state=0)
    for algo_name,config in algos.items():
        gs=GridSearchCV(config['model'],config['params'],cv=cv,return_train_score=False)
        gs.fit(X,y)
        scores.append({
            'model':algo_name,
            'best_score':gs.best_score_,
            'best_params':gs.best_params_
        })
        
    return pd.DataFrame(scores,columns=['model','best_score','best_params'])
        
find_best_model_using_gridsearchcv(X,y)
        
        

In [ ]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train, y_test=train_test_split(X,y,test_size=0.2,random_state=10)

from sklearn.tree import DecisionTreeRegressor
model=DecisionTreeRegressor()
model.fit(X_train,y_train)
model.score(X_test,y_test)

In [ ]:
def  predict_price(locality,sqft,bath,bedroom):
    loc_index=np.where(X.columns==locality)[0][0]
    x=np.zeros(len(X.columns))
    x[0]=sqft
    x[1]=bath
    x[2]=bedroom
    if loc_index>=0:
        x[loc_index]=1
    return lr_clf.predict([x])[0]

In [ ]:
predict_price('Hadapsar',1000,2,2)

In [ ]:
predict_price('Hadapsar',1000,3,3)

In [ ]:
predict_price('Koregaon Park',1000,3,3)

In [ ]:
import pickle
with open('bangalore_home_prices_model.pickle','wb') as f:
    pickle.dump(lr_clf,f)

In [ ]:
import json
columns={
    'data_columns': [col.lower() for col in X.columns]
}
with open("columns.json","w") as f:
    f.write(json.dumps(columns))